<br>

# Introdução


In [ ]:
#!pip3 install PyPDF2


In [ ]:
import os
import re
import time
import shutil
import requests
import numpy as np
import pandas as pd
from pathlib import Path
from PyPDF2 import PdfFileMerger, PdfFileReader


In [ ]:
from urllib.parse import urljoin
from urllib.parse import urlparse, urlunparse
import urllib.parse
import urllib.request


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from webdriver_manager.firefox import GeckoDriverManager


In [ ]:
from paths import *


In [ ]:
from dotenv import load_dotenv, find_dotenv, dotenv_values

# Credenciais
config = dotenv_values(find_dotenv(usecwd=True))

# Variaveis do Driver
os.environ['GH_TOKEN'] = config['GH_TOKEN']
CPF = config['CPF']
PASSWORD = config['PASSWORD']


<br>

## Create Driver


In [ ]:
def create_driver(download_path):
    # Parameters
    DRIVER_PATH = driver_path
    DRIVER_PATH = None
    HEADLESS = False
    #os.environ['WDM_SSL_VERIFY'] = '0'

    # Cria Driver
    global driver
    firefox_options = FirefoxOptions()
    firefox_options.headless = HEADLESS
    firefox_options.set_preference('browser.download.folderList', 2)
    firefox_options.set_preference(
        'browser.download.manager.showWhenStarting', False)
    firefox_options.set_preference('browser.download.dir', str(download_path))

    driver = webdriver.Firefox(
        service=Service(
            executable_path=GeckoDriverManager(path=DRIVER_PATH).install(),
            log_path=log_path / 'geckodriver.log',
        ),
        options=firefox_options,
    )

    # Add-ons Xpath
    xpath_path = adds_path / 'xpath.xpi'
    xpath_path = xpath_path.absolute().resolve()
    if not xpath_path.is_file():
        r = requests.get(
            'https://addons.mozilla.org/firefox/downloads/file/3588871/xpath_finder-1.0.2-fx.xpi')
        with open(xpath_path, 'wb') as f:
            f.write(r.content)
    driver.install_addon(str(xpath_path), temporary=True)


<br>

## Faz Login


In [ ]:
def fecha_popups():
    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # Hide Popup Tour
    try:
        tour = driver.find_element(
            By.XPATH, '//*[@class="hopscotch-bubble animated tour-portalTour"]//*[@class="hopscotch-actions"]//..')
        tour = tour.find_element(
            By.XPATH, './/*[@class="hopscotch-bubble-close hopscotch-close"]')
        tour.click()
    except:
        pass


In [ ]:
def get_login(cpf, password):
    # Get Url
    url = 'https://e.ambiente.sp.gov.br/atendimento/'
    driver.get(url)

    try:
        # Efetuar Login
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="info-usuario"]'))).click()

        # Pega o XPath do iframe e atribui a uma variável
        iframe = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))

        # Muda o foco para o iframe
        driver.switch_to.frame(iframe)

        # Entra com credenciais
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="j_username"]'))).send_keys(cpf)
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="j_password"]'))).send_keys(password)

        # Clica em entrar
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="pbEntrar"]'))).click()

        driver.switch_to.default_content()
    except:
        pass


<br>

## Go Pages


In [ ]:
def check_site():
    # ddd
    if driver.title != 'Portal de Atendimento':
        url = 'https://e.ambiente.sp.gov.br/atendimento/'
        driver.get(url)

In [ ]:
def open_sidebar():
    check_site()

    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    collapsed = driver.find_element(
        By.XPATH, '//aside[contains(@class, "abas")]').get_attribute('class')
    if collapsed == 'abas collapsed':
        driver.find_element(
            By.XPATH, '//*[@title="Exibir/Esconder menu"]').click()
    return 0


In [ ]:
def go_meusprocessos():
    # Open Sidebar
    open_sidebar()

    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # Clica em "Meus Processos"
    WebDriverWait(driver, 3).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@title="Meus Processos"]'))).click()

    # Muda o foco para o iframe
    iframe = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))
    driver.switch_to.frame(iframe)


In [ ]:
def go_aberturaprocessos():
    # Open Sidebar
    open_sidebar()

    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # Clica em "Processos Liberados"
    WebDriverWait(driver, 3).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@title="Abertura de Processos"]'))).click()

    # Muda o foco para o iframe
    iframe = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))
    driver.switch_to.frame(iframe)


In [ ]:
def go_processosliberados():
    # Open Sidebar
    open_sidebar()

    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # Clica em "Processos Liberados"
    WebDriverWait(driver, 3).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@title="Processos liberados"]'))).click()

    # Muda o foco para o iframe
    iframe = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))
    driver.switch_to.frame(iframe)
    

In [ ]:
def go_dadosusuario():
    # Open Sidebar
    open_sidebar()

    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # Clica em "Dados do Usuário"
    WebDriverWait(driver, 3).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@title="Dados do Usuário"]'))).click()

    # Muda o foco para o iframe
    iframe = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))
    driver.switch_to.frame(iframe)


<br>

## Meus Processos


In [ ]:
def get_meusprocessos():
    # Open Sidebar
    open_sidebar()

    # Go Page
    go_aberturaprocessos()
    time.sleep(3)
    go_meusprocessos()

    # Pega Table
    table = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="meusProcessos"]')))

    # Linhas
    WebDriverWait(table, 3).until(
        EC.presence_of_element_located((By.XPATH, './/tr')))
    rows = table.find_elements(By.XPATH, './/tr')

    # Loop
    list_head = []
    list_row = []
    list_processo = []
    for row in rows[:-2]:
        for cel in row.find_elements(By.XPATH, './/th'):
            list_head.append(cel.text)
        for cel in row.find_elements(By.XPATH, './/td'):
            list_row.append(cel.text)
            if cel.text == 'create_new_folder':
                list_processo.append(cel)

    # Pega Linhas de Interesse
    n_processos = len(rows[:-2])-1
    list_row = [x for x in list_row if x != '']
    list_row = np.array(list_row).reshape(n_processos, 5)

    # Dataframe
    df = pd.DataFrame(
        list_row,
        columns=list_head
    )
    df.drop(['Det.'], inplace=True, axis=1)
    return df


<br>

## Processos Liberados


In [ ]:
def get_processosliberados():
    # Open Sidebar
    open_sidebar()

    # Go Page
    go_aberturaprocessos()
    time.sleep(1)
    go_processosliberados()

    # Lista Processos
    tag = '//*[@class="sds-collapse-list sds-card liberacao-collapse-container"]//*[@class="sds-collapse-list__item"]'
    WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, tag)))
    list_process = driver.find_elements(By.XPATH, tag)

    list_process_text = []
    for process in list_process:
        list_process_text.append(process.text.split('\n'))

    # Create Dataframe
    df = pd.DataFrame(
        list_process_text,
        columns=['Nome', '-', 'Data de Liberação', '-',
                 'Acesso termina em', '-', 'Situação', '-']
    )
    # print(df.head())

    # Edit Dataframe
    df[['Processo', 'Interessado', 'Empresa']
       ] = df['Nome'].str.split(' - ', n=2, expand=True)

    df = df[['Processo', 'Interessado', 'Empresa',
             'Data de Liberação', 'Acesso termina em', 'Situação']].copy()

    df['Interessado'] = df['Interessado'].replace(
        {'Interessado: ': ''}, regex=True)

    # Result
    return df


<br>

## Select Process

In [ ]:
def select_processo(df_processosliberados):
    # Input
    valores = ''.join('{}'.format(list(df_processosliberados.index)))
    processo = int(input(
        f'Insira o número do índice do processo. O número deve estar entre {valores}: '))

    # Check se o processo está no índice
    while not(processo in list(df_processosliberados.index)):
        print('Número não consta no índice. Por favor, insira um número válido!')
        processo = int(input(
            f'Insira o número do índice do processo. O número deve estar entre {valores}: '))

    # Check situação
    processo_situacao = df_processosliberados.loc[processo, 'Situação']
    while processo_situacao != 'Acesso ativo':
        processo = int(input(
            f'Insira o número do índice do processo. O número deve estar entre {valores}: '))

    # Result
    processo_text = df_processosliberados.loc[processo, 'Processo'].replace(
        '.', '_').replace('/', '-')
    print(
        f'O processo escolhido foi "{processo_text}" e está com "{processo_situacao}"')
    return processo, processo_text


<br>

## get_file_list


In [ ]:
def get_file_list(processo):
    """
    url_1 é a primeira url... que, ao abrir, já muda...
    """
    # Open Sidebar
    open_sidebar()

    # Go Page
    go_aberturaprocessos()
    time.sleep(1)
    go_processosliberados()

    # Lista Processos
    tag = '//*[@class="sds-collapse-list sds-card liberacao-collapse-container"]//*[@class="sds-collapse-list__item"]'
    WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, tag)))
    list_process = driver.find_elements(By.XPATH, tag)

    # Minimiza todos os processos
    for process in list_process:
        try:
            tag = './/button[@class="sds-btn sds-btn--icon button-rotate"]'
            WebDriverWait(process, 1).until(
                EC.presence_of_element_located((By.XPATH, tag))).click()
            print('a')
        except:
            pass

    # Seleciona Processo
    WebDriverWait(list_process[processo], 1).until(EC.presence_of_element_located(
        (By.XPATH, './/button[@class="sds-btn sds-btn--icon"]'))).click()

    # Lista de todos os arquivos do processo
    tag = '//*[@id="secao-documentos{}"]//*[@class="sds-data-list"]//*[@class="sds-data-list__item"]'.format(
        processo)
    WebDriverWait(list_process[processo], 3).until(
        EC.presence_of_element_located((By.XPATH, tag)))
    files = driver.find_elements(By.XPATH, tag)

    # Loop
    list_name = []
    list_url = []
    for file in files:
        name = file.find_element(By.XPATH, './/span').text
        url = file.find_element(
            By.XPATH, './/*[@class="sds-link"]').get_attribute('href')
        list_name.append(name)
        list_url.append(url)

    # Dataframe
    df = pd.DataFrame(
        list(zip(list_name, list_url)),
        columns=['nome', 'url_1']
    )
    df['n_doc'] = df.index+1
    return df[['n_doc', 'nome', 'url_1']]


<br>

## get url redirects


In [ ]:
def get_url_redirects(url):
    """
    url_2 é a url que tem uma barra em cima!
    url_3 é o link pro PDF... e se ZIP, dá msg que não é possível dowload. Parâmetros Importantes!!!!   
    """
    # Get URL
    driver.get(url)
    time.sleep(2)

    while driver.execute_script('return document.readyState') != 'complete':
        time.sleep(2)

    # URL 2
    url1 = driver.current_url

    # URL 3
    driver.switch_to.default_content()
    iframe_1 = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, '//iframe[@id="content"]')))
    url2 = iframe_1.get_attribute('src')
    driver.switch_to.frame(iframe_1)

    # URL Download
    try:
        # Se for PDF
        iframe_2 = WebDriverWait(driver, 2).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@class="pdjJsFrameViewer"]')))
        url_temp = iframe_2.get_attribute('src')
        url_download = urllib.parse.unquote(
            url_temp, encoding='utf-8', errors='replace')
        url_download = url_download.replace(
            'go-docs-file-viewer/pdfjs/web/viewer.html?file=/', '')
        pdf = True
    except:
        # Não é PDF
        url_download = None
        pdf = False

    # Results
    return list([url1, url2, url_download, pdf])


<br>

## Merge PDFs


In [ ]:
def merge_files(processo_path):
    # Filename
    a = processo_path / f'{"Processo Unificado"}.pdf'
    
    # List Files
    list_files = list(processo_path.glob('*.pdf'))
    list_files = set(list_files)
    list_files.discard(a)

    # Lista documentos da Pasta
    list_dict = []
    for file in list_files:
        # print(file.name)
        n = file.name.split(' - ', 1)[0]
        list_dict.append({'id': n, 'nomearquivo': file})

    # Dataframe: files
    df = pd.DataFrame(list_dict)
    df['id'] = pd.to_numeric(df['id'])
    df.sort_values(by=['id'], inplace=True)
    df.reset_index(inplace=True)
    df.drop(['index'], inplace=True, axis=1)

    # Call the PdfFileMerger
    mergedObject = PdfFileMerger()
    for k, v in df.iterrows():
        bookmark = v['nomearquivo'].name
        mergedObject.append(
            PdfFileReader(
                processo_path / f'{v["nomearquivo"]}', 'rb'),
            bookmark
        )

    # Write all the files into a file which is named as shown below    
    mergedObject.write(str(a))
    return f'Doc Ok!'


<br>

## Download


In [ ]:
def is_download_finished(temp_folder):
    """
    https://stackoverflow.com/questions/34338897/python-selenium-find-out-when-a-download-has-completed
    """
    firefox_temp_file = sorted(Path(temp_folder).glob('*.part'))
    chrome_temp_file = sorted(Path(temp_folder).glob('*.crdownload'))
    downloaded_files = sorted(Path(temp_folder).glob('*.*'))
    if (len(firefox_temp_file) == 0) and \
       (len(chrome_temp_file) == 0) and \
       (len(downloaded_files) == 1):
        return True
    else:
        return False


In [ ]:
def download_selenium(path_search, path_move, out_filename):
    # ddd
    list_zipfiles = list(path_search.glob('*.*'))
    n_files = 0
    tentativa = 0
    while tentativa < 10 and n_files != 1:
        n_files = len(list_zipfiles)
        time.sleep(2)
        tentativa += 1

    test_download = is_download_finished(path_search)
    while not test_download:
        time.sleep(2)
        print(1)
        test_download = is_download_finished(path_search)

    if len(list_zipfiles) == 0:
        time.sleep(1)

    if len(list_zipfiles) == 1:
        # Se tem o arquivo
        # TODO: Add *.part ou *.crdownload

        teste = list_zipfiles[0].is_file()
        while teste:
            shutil.move(
                max([f for f in list_zipfiles], key=os.path.getctime), path_move / f'{out_filename}')
            time.sleep(2)

            teste = list_zipfiles[0].is_file()
        print(f'> Arquivo renomeado para "{out_filename}"')

    if len(list_zipfiles) > 1:
        time.sleep(1)
        print(f'Tem mais de arquivo no diretório "{path_search}"\nErro!!!!')
        raise Exception('Ajustar função "rename file"')


<br>

# Rull All


<br>

## Parte 1

In [ ]:
# test2 = is_download_finished(temp_path)
# print(test2)
# tentativa = 0
# while not test2 and tentativa < 10:
#     test2 = is_download_finished(temp_path)
#     print(test2)
#     time.sleep(2)
#     tentativa += 1


In [ ]:
# Cria Driver
create_driver(temp_path)


In [ ]:
# Login
get_login(CPF, PASSWORD)


In [ ]:
fecha_popups()


In [ ]:
# Meus Processos
#df_meusprocessos = get_meusprocessos()
#df_meusprocessos.head()


In [ ]:
# Processos Liberados
df_processosliberados = get_processosliberados()
df_processosliberados.head()


In [ ]:
# Select Process
processo, processo_text = select_processo(df_processosliberados)


In [ ]:
# Get File List
df = get_file_list(processo)
df.head()


In [ ]:
# Adjust Table: Função 1
df[['url_2', 'url_3', 'url_download', 'pdf']] = df.apply(
    lambda x: get_url_redirects(x['url_1']), axis=1, result_type='expand')


In [ ]:
# Create Directory
processo_path = data_path / processo_text
processo_path.mkdir(exist_ok=True)

# Escreve Tabela
df.to_csv(
    processo_path / 'tab_arquivos.csv',
    index=False
)


<br>

## Parte 2

In [ ]:
# Create Directory
processo_path = data_path / processo_text
processo_path.mkdir(exist_ok=True)

# Lê Tabela
df = pd.read_csv(processo_path / 'tab_arquivos.csv')

# Filter
mask = (df['n_doc'] >= 30) & (df['n_doc'] <= 20000)
df = df[mask].copy()

# Results
df.head()


In [ ]:
# https://stackoverflow.com/questions/29563335/how-do-i-load-session-and-cookies-from-selenium-browser-to-requests-library-in-p
# Get cookies
cookies = driver.get_cookies()

# Create session from cookies
s = requests.Session()
for cookie in cookies:
    s.cookies.set(cookie['name'], cookie['value'])


In [ ]:
# Loop thought table
for k, v in df.iterrows():
    # Parametros
    n_doc = v['n_doc']
    url_download = v['url_download']
    url_2 = v['url_2']
    nome = v['nome']
    pdf = v['pdf']

    # Se for PDF
    if pdf:
        print(f'Download do arquivo {n_doc} - {nome}\n{url_download}\n')

        # ddddddd
        r = s.get(url_download, stream=True)
        try:
            text = r.content.decode('utf-8')
            print(text)
        except:
            text = 'É PDF!'

        # dddd
        if text != 'Não foi possível recuperar a liberação externa para este documento':
            # Salva Arquivo
            time.sleep(5)
            
            # Adjust Nome
            nome = nome.replace('.pdf ', ' ')
            nome = re.sub('[\\/:"*?<>|]+', '', nome)
            
            filename = processo_path / f'{n_doc} - {nome}.pdf'
            if r.status_code == 200:
                with open(filename, 'wb') as f:
                    for chunk in r:
                        f.write(chunk)

        elif text == 'Não foi possível recuperar a liberação externa para este documento':
            print('Arrumar esta merda!')

    elif not pdf:
        print(f'Download do arquivo {n_doc} - {nome}\n{url_2}\n')

        # sss
        driver.get(url_2)
        driver.implicitly_wait(20)

        # Retorna para a janela principal (fora do iframe)
        driver.switch_to.default_content()

        # Muda o foco para o iframe
        iframe = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]')))
        driver.switch_to.frame(iframe)
        time.sleep(1)

        # Clica em Download
        WebDriverWait(driver, 20).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="btnDownload"]'))).click()
        time.sleep(20)

        # Move
        nome = nome.replace('.zip ', ' ').replace('.ZIP ', ' ')
        download_selenium(temp_path, processo_path, f'{n_doc} - {nome}.zip')


<br>

## Parte 3


In [ ]:
# Merge PDF
merge_files(processo_path)


In [ ]:
# Fecha Driver
driver.quit()

In [ ]:
# 
temp_path.rmdir()


In [ ]:
#shutil.rmtree(temp_path)